<a href="https://colab.research.google.com/github/vlozg/speech_hmm/blob/main/Test_DiagHMM_004.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LƯU Ý: NOTEBOOK NÀY CHỈ DÙNG ĐỂ SHOW KẾT QUẢ CHẠY, KHÔNG NÊN CHẠY LẠI NOTEBOOK NÀY 
(VÌ TRONG NÀY CÓ CODE LƯU LẠI PRETRAINED MODEL LÊN DRIVE SẼ BỊ XÓA)

# Speech to text with HMM

- **Bài toán**: Chuyển giọng nói thành văn bản
    - **Input**: Đoạn ghi âm chứa nội dung là các số từ 0 đến 9
    - **Output**: Phân lớp của đoạn ghi âm

# Các biến thiết lập cho thử nghiệm

In [ ]:
n_mfcc_ceptrum = 12
n_delta_features = 1
fsdd_split = 0.3
wolfram_split = 0.3
experiment_id = '004'

In [ ]:
n_mfcc_features = n_mfcc_ceptrum * (1+n_delta_features)
n_mfcc_features

24

# Import và cài đặt thư viện

In [ ]:
# cài lib. note: cài xong phải restart runtime
!pip install pydub
!pip install pomegranate

In [ ]:
# Xác thực google để upload/download qua google drive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Quản lý file, folder
import os
from shutil import copyfile, rmtree
import random

# Xử lý audio
import librosa
import librosa.display
from scipy.io import wavfile

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import pomegranate # Thư viện cho mô hình xác suất
from pomegranate import *

# Tải dữ liệu và lấy xác thực Google

Dữ liệu dùng để huấn luyện và đánh giá, còn xác thực google thì dùng để upload/download mô hình trên drive.

In [ ]:
%%capture
# download wolfram
if not os.path.isfile('./dataset_1_wolfram.zip'):
  !gdown --id 115tIAitBNeJC0DwrP-ZyJ6RS3TyWN0qD
  !unzip -o dataset_1_wolfram.zip

# dowload FSDD
if not os.path.isfile('./dataset_2_FSDD.zip'):
  !gdown --id 1Ua9zlPBc0Fv4xGHSQTb7eIvUh_dqFI6P
  !unzip -o dataset_2_FSDD.zip

# download self recorded audio
!gdown --id 1lH_k1AYMVlJvodtZdD7OK2zkdPXxlW9i

In [ ]:
# Lấy xác thực google để upload/download file
auth.authenticate_user()
gauth =  GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Hàm xử lý âm thanh

In [ ]:
def scaleAddNoise(wave):
  return ((wave - wave.mean()) / wave.std()) + 0.2*np.random.normal(size=wave.shape)

def read_process_audio(full_audio_path):
  # Bắt buộc âm thanh đọc vào phải cùng sample rate
  sample_rate, wave =  wavfile.read(full_audio_path)
  if sample_rate != 16000:
    wave, sample_rate =  librosa.load(full_audio_path, sr=16000) # Hàm đọc của librosa quá chậm, scipy nhanh gấp 5-10 lần
  else:
    wave=wave/32768 # Chuẩn hóa về số thực
  wave = np.pad(wave, (2000,2000), 'constant', constant_values=(0.0,0.0))
  wave = scaleAddNoise(wave)
  return wave, sample_rate

def extract_mfcc(wave, sample_rate):
  mfccs = librosa.feature.mfcc(y=wave, sr=sample_rate, n_mfcc=n_mfcc_ceptrum+2, n_mels=40, lifter=40)[2:,:].T/800
  return mfccs

def mfcc_delta_features(mfcc, order):
  if order==0:
    return mfcc
  dmfcc = librosa.feature.delta(mfcc, order=order)
  return dmfcc

def full_mfcc_from_file(full_audio_path):
  wave, sr = read_process_audio(full_audio_path)
  mfccs = extract_mfcc(wave, sr)
  full_mfccs = mfccs
  if n_delta_features >= 1:
    mfccs_d1 = mfcc_delta_features(mfccs, 1)
    full_mfccs = np.hstack([full_mfccs,mfccs_d1])
  if n_delta_features >= 2:
    mfccs_d2 = mfcc_delta_features(mfccs, 2)
    full_mfccs = np.hstack([full_mfccs,mfccs_d2])
  return full_mfccs

# Hàm bổ trợ

In [ ]:
def buildDataSet(dir):
    # Filter out the wav audio files under the dir
    fileList = [f for f in os.listdir(dir) if os.path.splitext(f)[1] == '.wav']
    dataset = {}
    for fileName in fileList:
        tmp = fileName.split('.')[0]
        label = tmp.split('_')[1]

        # label = filename.split('_')[0]
        feature = full_mfcc_from_file(dir+fileName)
        if label not in dataset.keys():
            dataset[label] = []
            dataset[label].append(feature)
        else:
            exist_feature = dataset[label]
            exist_feature.append(feature)
            dataset[label] = exist_feature
    return dataset

def makeTrainTestDir(mainFolder, filenames, rate):
    paths = [f'{mainFolder}',
             f'{mainFolder}/train_audio',
             f'{mainFolder}/test_audio']

    for path in paths:
        try:
            os.mkdir(path)
        except:
            rmtree(path)
            os.mkdir(path)

    random.seed(1)
    random.shuffle(filenames)
    splitPoint = int(len(filenames)*rate)
    trainFilenames = filenames[:splitPoint]
    testFilenames = filenames[splitPoint:]

    for filename in trainFilenames:
        copyfile(filename[0], f"{paths[1]}/{filename[1]}")

    for filename in testFilenames:
        copyfile(filename[0], f"{paths[2]}/{filename[1]}")

# Setup thư mục chứa data

In [ ]:
def formatFilenameFSDD(dir):
    filenames = []
    count = 0
    for filename in os.listdir(dir):
        tmp = str(count) + '_' + filename.split('_')[0] + '.wav'
        filenames.append((f"{dir}/{filename}", tmp))
        count += 1
    return filenames

filenames = formatFilenameFSDD('./dataset_2_FSDD')
makeTrainTestDir('fsdd', filenames, fsdd_split)

In [ ]:
def formatFilenameWolfram(dir):
    filenames = []
    count = 0
    folders = os.listdir(dir)
    for folder in folders:
        for filename in os.listdir(f'{dir}/{folder}'):
            tmp = str(count) + '_' + folder.split('_')[0] + '.wav'
            filenames.append((f"{dir}/{folder}/{filename}", tmp))
            count += 1

    return filenames

filenames = formatFilenameWolfram('./dataset_1_wolfram')
makeTrainTestDir('wolfram', filenames, wolfram_split)

# Huấn luyện mô hình (có thể bỏ qua vì mô hình đã save trên drive)

## Hàm train mô hình

In [ ]:
def Generate_DiagGMM(full_fset, n_features, n_states=5, n_cmps=3):
  means = np.mean(full_fset, axis=0)
  stds = np.std(full_fset, axis=0)
  # initial values for all gaussian components
  np.random.seed(None)
  dist_init = np.random.random((n_states, n_cmps, n_features, 2))
  dist_init[..., 0] -= 0.5  # center means to 0.0
  #for feat_i in range(n_features):
      # random init mean in range [-std, std)
      #dist_init[..., feat_i, 0] *= 2 * stds[feat_i]
      #dist_init[..., feat_i, 0] += means[feat_i]
      # random init std in range 1std/n_components
      #dist_init[..., feat_i, 1] *= stds[feat_i] / n_cmps

  dists = tuple(
      GeneralMixtureModel(list(
          IndependentComponentsDistribution(tuple(
              NormalDistribution(*dist_init[state_i, cmp_i, feat_i, :])
              for feat_i in range(n_features)
          ))
          for cmp_i in range(n_cmps)
      ))
      if n_cmps > 1 else
      IndependentComponentsDistribution(tuple(
          NormalDistribution(*dist_init[state_i, 0, feat_i, :])
          for feat_i in range(n_features)
      ))
      for state_i in range(n_states)
  )
  return dists

In [ ]:
# Generate progressive HMM model
def left_right_GMMHMM(seed_sample, x_dim, n_states=10, n_modals=9, random=0):
  rng = np.random.RandomState(random)
  if random!=0:
    init_prob = lambda: rng.rand(1)[0]
    
  model = HiddenMarkovModel()
  states = [State( state_model, name=f"H{i}" ) for i, state_model in enumerate(Generate_DiagGMM(seed_sample, x_dim, n_states, n_modals))]
  model.add_states(states)
  model.add_transition(model.start, states[0], 1)
  for i in range(n_states-1):
    model.add_transition(states[i], states[i], 0.5)
    model.add_transition(states[i], states[i+1], 0.5)
  model.add_transition(states[n_states-1], states[n_states-1], 0.5)
  model.add_transition(states[n_states-1], model.end, 0.5)
  model.bake()
  return model

In [ ]:
def train_GMMHMM(dataset, input_dim, n_hidden_state, n_gauss_modal):
    GMMHMM_Models = dict()

    for label in dataset.keys():
        print(f"Training model detect {label}")
        model = left_right_GMMHMM(dataset[label][0], input_dim, n_hidden_state, n_gauss_modal)
        model.fit(dataset[label], verbose=True, multiple_check_input=False)  # get optimal parameters
        GMMHMM_Models[label] = model

    return GMMHMM_Models

## Hàm test mô hình

In [ ]:
# test model
def evaluateModel(testDataSet, model):
    if (len(testDataSet) == 0):
      return
    score_cnt = 0
    total = 0
    for label in testDataSet.keys():
        features = testDataSet[label]
        total += len(features)
        for sample in features:
            scoreList = {}
            for model_label in model.keys():
                score = model[model_label].log_probability(sample)
                scoreList[model_label] = score
            predict = max(scoreList, key=scoreList.get)
            if predict == label:
                score_cnt += 1
        print(f"{label}: ({score_cnt}/{total})")

    print("Final recognition rate is %.2f"%(100.0*score_cnt/total), "%")

## Đọc và tiền xử lý data

In [ ]:
# prepare data for training
master_path = 'fsdd'

trainDir = master_path + '/train_audio/'
trainDataSet_fsdd = buildDataSet(trainDir)
print("Finish prepare the training data")

# prepare data for testing
testDir = master_path + '/test_audio/'
testDataSet_fsdd = buildDataSet(testDir)
print("Finish prepare the test data")

Finish prepare the training data
Finish prepare the test data


In [ ]:
# prepare data for training
master_path = 'wolfram'

trainDir = master_path + '/train_audio/'
trainDataSet_wolfram = buildDataSet(trainDir)
print("Finish prepare the training data")

# prepare data for testing
testDir = master_path + '/test_audio/'
testDataSet_wolfram = buildDataSet(testDir)
print("Finish prepare the test data")

Finish prepare the training data
Finish prepare the test data


In [ ]:
trainDataSet = trainDataSet_fsdd
if wolfram_split > 0:
  for label in trainDataSet_fsdd.keys():
    trainDataSet[label] = trainDataSet_fsdd[label] + trainDataSet_wolfram[label]

## **Mô hình 1**
- 10 hiddent states
- Multivariate Diagonal Gauss cho emission probs

In [ ]:
%%time
# train
hmmModels = train_GMMHMM(trainDataSet, n_mfcc_features,10,1)
print("Finish training of the GMM_HMM models for digits 0-9")

Training model detect 1
[1] Improvement: 21806313.5525234	Time (s): 0.3879
[2] Improvement: 106513.91975777433	Time (s): 0.3641
[3] Improvement: 9597.943589705974	Time (s): 0.3733
[4] Improvement: 4173.348130477825	Time (s): 0.3883
[5] Improvement: 2427.5558303983416	Time (s): 0.3806
[6] Improvement: 2419.0401605938096	Time (s): 0.3875
[7] Improvement: 2407.271540256217	Time (s): 0.3742
[8] Improvement: 1738.468054822646	Time (s): 0.371
[9] Improvement: 1205.5506462084595	Time (s): 0.3748
[10] Improvement: 825.0667577118147	Time (s): 0.384
[11] Improvement: 640.5523672066629	Time (s): 0.3699
[12] Improvement: 438.4342001886107	Time (s): 0.3787
[13] Improvement: 540.5423341682181	Time (s): 0.3799
[14] Improvement: 3020.6389290988445	Time (s): 0.3717
[15] Improvement: 1529.449076956138	Time (s): 0.3726
[16] Improvement: 439.7900805573445	Time (s): 0.3722
[17] Improvement: 271.7070511667989	Time (s): 0.3668
[18] Improvement: 245.6269399591256	Time (s): 0.3777
[19] Improvement: 189.9054384

In [ ]:
print("Evaluate on train set")
evaluateModel(trainDataSet, hmmModels)
print("Evaluate on rest of FSDD set")
evaluateModel(testDataSet_fsdd, hmmModels)
print("Evaluate on wolfram test set")
evaluateModel(testDataSet_wolfram, hmmModels)

Evaluate on train set
1: (553/837)
2: (951/1582)
6: (1442/2428)
8: (2054/3223)
0: (2649/4018)
7: (3144/4807)
9: (3718/5591)
5: (4334/6403)
3: (4715/7196)
4: (5334/7999)
Final recognition rate is 66.68 %
Evaluate on rest of FSDD set
0: (158/211)
1: (290/419)
3: (414/637)
9: (544/839)
7: (720/1073)
4: (904/1287)
5: (1087/1490)
8: (1270/1687)
2: (1422/1901)
6: (1502/2100)
Final recognition rate is 71.52 %
Evaluate on wolfram test set
4: (1237/1655)
8: (2368/3315)
1: (3388/4940)
2: (4241/6654)
6: (5224/8278)
0: (6467/9948)
3: (7242/11593)
7: (8290/13247)
5: (9536/14889)
9: (10768/16567)
Final recognition rate is 65.00 %


In [ ]:
# lưu lại model
for model_label in hmmModels.keys():
  file = drive.CreateFile({'title': f'hmm[{model_label}]_{n_mfcc_features}_10_1[{experiment_id}].json', 'parents': [{'id': '1QPUr4vwYHu3n9iH3iQmnvDUt2Dgx4V3Y'}]})
  file.SetContentString(hmmModels[model_label].to_json())
  file.Upload()

## **Mô hình 2**
- 5 hiddent states
- Multivariate Diagonal Gauss cho emission probs

In [ ]:
%%time
# train
hmmModels = train_GMMHMM(trainDataSet, n_mfcc_features,5,1)
print("Finish training of the GMM_HMM models for digits 0-9")

Training model detect 1
[1] Improvement: 3098964.1844168715	Time (s): 0.2051
[2] Improvement: 55143.138193891384	Time (s): 0.2075
[3] Improvement: 88041.01842301781	Time (s): 0.2135
[4] Improvement: 61759.44372064457	Time (s): 0.2322
[5] Improvement: 19756.48109965818	Time (s): 0.2134
[6] Improvement: 4490.971274620388	Time (s): 0.2104
[7] Improvement: 922.9937294323463	Time (s): 0.2132
[8] Improvement: 290.34028268931434	Time (s): 0.1983
[9] Improvement: 104.17934950068593	Time (s): 0.2121
[10] Improvement: 57.39874282036908	Time (s): 0.1992
[11] Improvement: 96.50858260504901	Time (s): 0.2104
[12] Improvement: 46.165363773936406	Time (s): 0.2112
[13] Improvement: 13.034318165620789	Time (s): 0.1998
[14] Improvement: 5.759363709250465	Time (s): 0.2157
[15] Improvement: 1.4812587946653366	Time (s): 0.2114
[16] Improvement: 0.5463128078263253	Time (s): 0.2105
[17] Improvement: 0.319079568143934	Time (s): 0.1992
[18] Improvement: 0.3179084414150566	Time (s): 0.2184
[19] Improvement: 0.57

In [ ]:
print("Evaluate on train set")
evaluateModel(trainDataSet, hmmModels)
print("Evaluate on rest of FSDD set")
evaluateModel(testDataSet_fsdd, hmmModels)
print("Evaluate on wolfram test set")
evaluateModel(testDataSet_wolfram, hmmModels)

Evaluate on train set
1: (449/837)
2: (451/1582)
6: (936/2428)
8: (1120/3223)
0: (1586/4018)
7: (2001/4807)
9: (2220/5591)
5: (2797/6403)
3: (3409/7196)
4: (3918/7999)
Final recognition rate is 48.98 %
Evaluate on rest of FSDD set
0: (113/211)
1: (225/419)
3: (438/637)
9: (444/839)
7: (589/1073)
4: (767/1287)
5: (951/1490)
8: (967/1687)
2: (967/1901)
6: (1045/2100)
Final recognition rate is 49.76 %
Evaluate on wolfram test set
4: (1026/1655)
8: (1436/3315)
1: (2299/4940)
2: (2302/6654)
6: (3254/8278)
0: (4189/9948)
3: (5369/11593)
7: (6265/13247)
5: (7410/14889)
9: (7922/16567)
Final recognition rate is 47.82 %


In [ ]:
# lưu lại model
for model_label in hmmModels.keys():
  file = drive.CreateFile({'title': f'hmm[{model_label}]_{n_mfcc_features}_5_1[{experiment_id}].json', 'parents': [{'id': '1QPUr4vwYHu3n9iH3iQmnvDUt2Dgx4V3Y'}]})
  file.SetContentString(hmmModels[model_label].to_json())
  file.Upload()

## **Mô hình 3**
- 3 hiddent states
- Multivariate Diagonal Gauss cho emission probs

In [ ]:
%%time
# train
hmmModels = train_GMMHMM(trainDataSet, n_mfcc_features,3,1)
print("Finish training of the GMM_HMM models for digits 0-9")

Training model detect 1
[1] Improvement: 3012059.8236490446	Time (s): 0.1423
[2] Improvement: 388.3996129031293	Time (s): 0.1379
[3] Improvement: 84520.4049551303	Time (s): 0.1365
[4] Improvement: 28280.959373764927	Time (s): 0.1364
[5] Improvement: 576.940775990719	Time (s): 0.146
[6] Improvement: 155.34674619953148	Time (s): 0.1447
[7] Improvement: 78.35351359425113	Time (s): 0.1468
[8] Improvement: 59.98937802272849	Time (s): 0.1474
[9] Improvement: 168.28692078427412	Time (s): 0.1427
[10] Improvement: 3908.5448683095165	Time (s): 0.1379
[11] Improvement: 5920.782896505902	Time (s): 0.1373
[12] Improvement: 1872.7308531182352	Time (s): 0.1391
[13] Improvement: 1433.8705958607607	Time (s): 0.1521
[14] Improvement: 710.4456543661654	Time (s): 0.1396
[15] Improvement: 286.87958235898986	Time (s): 0.1426
[16] Improvement: 160.00762565783225	Time (s): 0.1379
[17] Improvement: 42.18375341594219	Time (s): 0.1373
[18] Improvement: 35.19232236617245	Time (s): 0.1441
[19] Improvement: 55.7821

In [ ]:
print("Evaluate on train set")
evaluateModel(trainDataSet, hmmModels)
print("Evaluate on rest of FSDD set")
evaluateModel(testDataSet_fsdd, hmmModels)
print("Evaluate on wolfram test set")
evaluateModel(testDataSet_wolfram, hmmModels)

Evaluate on train set
1: (98/837)
2: (389/1582)
6: (862/2428)
8: (870/3223)
0: (1175/4018)
7: (1191/4807)
9: (1226/5591)
5: (1852/6403)
3: (2294/7196)
4: (2319/7999)
Final recognition rate is 28.99 %
Evaluate on rest of FSDD set
0: (43/211)
1: (50/419)
3: (231/637)
9: (238/839)
7: (239/1073)
4: (247/1287)
5: (412/1490)
8: (412/1687)
2: (549/1901)
6: (641/2100)
Final recognition rate is 30.52 %
Evaluate on wolfram test set
4: (57/1655)
8: (81/3315)
1: (267/4940)
2: (886/6654)
6: (1872/8278)
0: (2534/9948)
3: (3445/11593)
7: (3479/13247)
5: (4751/14889)
9: (4812/16567)
Final recognition rate is 29.05 %


In [ ]:
# lưu lại model
for model_label in hmmModels.keys():
  file = drive.CreateFile({'title': f'hmm[{model_label}]_{n_mfcc_features}_3_1[{experiment_id}].json', 'parents': [{'id': '1QPUr4vwYHu3n9iH3iQmnvDUt2Dgx4V3Y'}]})
  file.SetContentString(hmmModels[model_label].to_json())
  file.Upload()

# Đọc vào model đã save

In [ ]:
# download pre-trained models
folder_id  = '1QPUr4vwYHu3n9iH3iQmnvDUt2Dgx4V3Y'
file_list = drive.ListFile({'q': "'{}' in parents and trashed=false".format(folder_id)}).GetList()
for i, file1 in enumerate(sorted(file_list, key = lambda x: x['title']), start=1):
  if (file1['title'][:3] != "hmm" or f"[{experiment_id}]" not in file1['title']):
    continue
  print('Downloading {} from GDrive ({}/{})'.format(file1['title'], i, len(file_list)))
  file1.GetContentFile(file1['title'])

In [ ]:
# Đọc model và tạo classifier `digit_clf`
# model_id có định dạng 13_<số state>_<số mixture>[số stt]
def load_model_as_clf(model_id):
  hmmModels = []
  for model_label in range(10):
    with open(f'hmm[{model_label}]_{model_id}.json','rt') as f:
      hmmModels.append(HiddenMarkovModel().from_json(f.read()))

  digit_clf = BayesClassifier(hmmModels)
  return digit_clf

# Thử model

### Kiểm thử với tập dữ liệu Wolfram

In [ ]:
%%time
testDataset = buildDataSet('./wolfram/test_audio/')

In [ ]:
def final_test(testDataset, digit_clf):
  true_cnt = 0
  total = 0
  for label in testDataset.keys():
      iter_cnt = 0
      features = testDataset[label]
      total += len(features)
      for sample in features:
          pred = digit_clf.predict(np.array([sample]))[0]
          if pred == int(label):
              iter_cnt += 1
      true_cnt += iter_cnt
      print(f"{label}: {iter_cnt}/{len(features)} ({iter_cnt/len(features)})")

  print("Final recognition rate is %.2f"%(100.0*true_cnt/total), "%")

In [ ]:
final_test(testDataset, load_model_as_clf(f"{n_mfcc_features}_10_1[{experiment_id}]"))

In [ ]:
final_test(testDataset, load_model_as_clf(f"{n_mfcc_features}_5_1[{experiment_id}]"))

In [ ]:
final_test(testDataset, load_model_as_clf(f"{n_mfcc_features}_3_1[{experiment_id}]"))